In [ ]:
# default_exp datastruct.possessions

In [ ]:
from IPython.core.debugger import set_trace
from IPython.utils import traitlets as _traitlets

/tmp/ipykernel_592056/2943078347.py:2: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  from IPython.utils import traitlets as _traitlets


<h1><center> Possessions Data Structure </center></h1>

In this module, we provide a class mapping between the `Possessions` document in `mongoDB` and python.


The `Possessions` document has a nested structure with:
+ some metadata: `gameId`, `possNumber`, `target` and a unique document `_id`
+ a List of events stored in the `sequence` field

The strategy therefore is to define first an `event` `mongoengine.EmbeddedDocument` and then the `Possession` `mongoengine.Document`

# `Event` Embedded Document

In [ ]:
# export

import datetime

import mongoengine
import pandas as pd
from fastcore.basics import *

from footSeq.config.localconfig import CONFIG


class Event(mongoengine.EmbeddedDocument):
    "Event mapping class to store event info stored in the `sequence` field"
    
    ## events
    event_id = mongoengine.IntField(db_field="event_id")
    orig_event_id = mongoengine.StringField(db_field="eventId")
    
    ## player
    player_id = mongoengine.IntField(db_field="player_id", required=False)
    player_name = mongoengine.StringField(
        db_field="player_name", required=False, default="Unknown"
    )
    
    ## team
    team_id = mongoengine.IntField(db_field="team_id")
    team_name = mongoengine.StringField(db_field="team_name")
    
    ## possession
    possession_id = mongoengine.IntField(db_field="poss_id", required=True)
    possession_name = mongoengine.StringField(db_field="possession_name", required=True)
    possession_team_id = mongoengine.IntField(
        db_field="possession_team_id", required=True
    )
    possession_number = mongoengine.IntField(
        db_field="possession_number", required=True
    )
    seconds_since_poss = mongoengine.FloatField(
        db_field="seconds_since_poss", min_value=0.0, required=True
    )
    standart_name = mongoengine.StringField(db_field="standart_name", required=True)
    
    ## time
    period_id = mongoengine.IntField(
        db_field="period_id", required=True, min_value=1, max_value=2
    )
    time_seconds = mongoengine.FloatField(db_field="time_seconds", required=True)

    ## attack types
    attack_status_name = mongoengine.StringField(
        db_field="attack_status_name", required=True
    )
    attack_type_name = mongoengine.StringField(
        db_field="attack_type_name", required=True
    )
    attack_team_id = mongoengine.StringField(db_field="attack_team_id", required=True)
    
    ## actions
    action_id = mongoengine.IntField(db_field="action_id", required=True)
    type_name = mongoengine.StringField(db_field="type_name", required=True)
    result_id = mongoengine.IntField(
        db_field="result_id", required=True, min_value=-1, max_value=1
    )
    result_name = mongoengine.StringField(db_field="result_name", required=True)
    generic_action_type_id = mongoengine.IntField(
        db_field="generic_action_type_id", required=True
    )
    generic_action_type_name = mongoengine.StringField(
        db_field="generic_action_type_name", required=True
    )
    action_subtype_name = mongoengine.StringField(db_field="action_subtype_name", required=True)
    touches = mongoengine.StringField(db_field="touches", required=True)
    shot_type = mongoengine.StringField(db_field="shot_type", required=True)
    shot_handling = mongoengine.StringField(db_field="shot_handling", required=True)
    
    ## extra info
    under_pressure = mongoengine.BooleanField(db_field="under_pressure", required=True)
    high_speed = mongoengine.BooleanField(db_field="high_speed", required=True)
    body_name = mongoengine.StringField(
        db_field="body_name", required=False, default="Unknown"
    )
    
    ## we allow extra meters to adjust pitch coordinates
    start_x = mongoengine.FloatField(db_field="start_x", min_value=-1, max_value=107)
    start_y = mongoengine.FloatField(db_field="start_y", min_value=-1, max_value=70)
    end_x = mongoengine.FloatField(db_field="end_x", min_value=-1, max_value=107)
    end_y = mongoengine.FloatField(db_field="end_y", min_value=-1, max_value=70)
    
    ## goal gate
    gate_x = mongoengine.FloatField(db_field="gate_x", required=False)
    gate_y = mongoengine.FloatField(db_field="gate_y", required=False)
    
    ## opponent
    opponent_position_id = mongoengine.IntField(db_field="opponent_position_id", required=False)
    opponent_id = mongoengine.IntField(db_field="opponent_id", required=False) 
    opponent_team_name = mongoengine.StringField(db_field="opponent_team_name", required=False) 
    opponent_team_id = mongoengine.IntField(db_field="opponent_team_id", required=False)
    opponent_name = mongoengine.StringField(db_field="opponent_name", required=False)

    ## binary flags
    is_poss_team = mongoengine.BooleanField(db_field="is_poss_team", required=True)
    is_att_team = mongoengine.BooleanField(db_field="is_att_team", required=True)


def _convert_event(eve: Event):
    return pd.DataFrame({x: eve[x] for x in eve}, index=[0])

# The `Possession` Document

In [ ]:
# export


class Possession(mongoengine.Document):
    """
    Store the `Possession` document info

    Attributes
    ----------
    _id : str
        unique document identifier
    gameId : int
        game-identifier
    possNumber : int
        possession number
    target : str
        whether the possession ended up in a `goal` or `no-goal`
    "sequence": List[Event]
        list of event information

    """

    game_id = mongoengine.IntField(db_field="gameId", required=True)
    possession_number = mongoengine.IntField(db_field="possNumber", required=True)
    target = mongoengine.StringField(db_field="target", choices=("goal", "no-goal"))
    sequence = mongoengine.EmbeddedDocumentListField(Event)

    meta = {
        "db_alias": "inStat",
        "collection": CONFIG["connections"]["inStat"]["possessions"],
    }

    def _format_poss(_raw_obj):
        return pd.DataFrame(
            {
                "gameId": _raw_obj.game_id,
                "possessionNumber": _raw_obj.possession_number,
                "target": _raw_obj.target,
                "sequence": listify(
                    pd.concat([_convert_event(eve) for eve in _raw_obj.sequence])
                    .reset_index(drop=True)
                    .sort_values(["period_id", "time_seconds"])
                ),
            },
            index=[0],
        )

    @classmethod
    def get_game_poss(cls, gm_id: int, poss_nbr: int) -> pd.DataFrame:
        "extract information for a given possession in a game"
        return cls._format_poss(
            cls.objects(game_id=gm_id, possession_number=poss_nbr).first()
        )

    @classmethod
    def get_all_game_poss(cls, gm_id: int):
        "extract all possessions for a game"
        return (
            pd.concat(
                [cls._format_poss(_raw_obj) for _raw_obj in cls.objects(game_id=gm_id)]
            )
            .reset_index(drop=True)
            .sort_values(["possessionNumber"])
        )

In [ ]:
from footSeq.config.mongo import mongo_init

mongo_init("prod_atlas")

poss = Possession.get_all_game_poss(gm_id=1998205)

In [ ]:
poss

,gameId,possessionNumber,target,sequence
6,1998205,1,no-goal,event_id orig_event_id player_id ...
1,1998205,2,no-goal,event_id orig_event_id player_id ...
2,1998205,3,no-goal,event_id orig_event_id player_id ...
0,1998205,4,no-goal,event_id orig_event_id player_id ...
7,1998205,5,no-goal,event_id orig_event_id player_id ...
...,...,...,...,...
147,1998205,154,no-goal,event_id orig_event_id player_id ...
150,1998205,155,no-goal,event_id orig_event_id player_id ...
151,1998205,156,no-goal,event_id orig_event_id player_id ...
145,1998205,157,no-goal,event_id orig_event_id player_id ...


In [ ]:
poss.sequence[0]

,event_id,orig_event_id,player_id,player_name,team_id,team_name,possession_id,possession_name,possession_team_id,possession_number,...,end_y,gate_x,gate_y,opponent_position_id,opponent_id,opponent_team_name,opponent_team_id,opponent_name,is_poss_team,is_att_team
0,5184,1998205_5184,70856,Hakan Calhanoglu,151,Inter Milan,1998205___4___5184___5219___no-goal,Ball possession,151,4,...,15.7,0.00,0.00,42,94779,Inter Milan,151,Milan Skriniar,True,True
1,5185,1998205_5185,94779,Milan Skriniar,151,Inter Milan,1998205___4___5184___5219___no-goal,Ball possession,151,4,...,30.9,0.00,0.00,32,4320,Inter Milan,151,Stefan de Vrij,True,True
2,5186,1998205_5186,4320,Stefan de Vrij,151,Inter Milan,1998205___4___5184___5219___no-goal,Ball possession,151,4,...,30.9,0.00,0.00,None,None,None,None,None,True,True
3,5187,1998205_5187,4320,Stefan de Vrij,151,Inter Milan,1998205___4___5184___5219___no-goal,Ball possession,151,4,...,15.5,0.00,0.00,42,94779,Inter Milan,151,Milan Skriniar,True,True
4,5188,1998205_5188,94779,Milan Skriniar,151,Inter Milan,1998205___4___5184___5219___no-goal,Ball possession,151,4,...,15.5,0.00,0.00,None,None,None,None,None,True,True
5,5189,1998205_5189,94779,Milan Skriniar,151,Inter Milan,1998205___4___5184___5219___no-goal,Ball possession,151,4,...,2.6,0.00,0.00,33,26140,Inter Milan,151,Marcelo Brozovic,True,True
6,5190,1998205_5190,26140,Marcelo Brozovic,151,Inter Milan,1998205___4___5184___5219___no-goal,Ball possession,151,4,...,27.3,0.00,0.00,46,1693,Inter Milan,151,Alexis Alehandru Sanchez,True,True
7,5191,1998205_5191,1693,Alexis Alehandru Sanchez,151,Inter Milan,1998205___4___5184___5219___no-goal,End,151,4,...,21.2,0.00,0.00,None,None,None,None,None,True,True
8,5192,1998205_5192,3898,Diego Roberto Godin Leal,272,Cagliari,1998205___4___5184___5219___no-goal,Ball possession,151,4,...,27.3,0.00,0.00,46,1693,Inter Milan,151,Alexis Alehandru Sanchez,False,False
9,5194,1998205_5194,204266,Nicolo Barella,151,Inter Milan,1998205___4___5184___5219___no-goal,Ball possession,151,4,...,14.6,0.00,0.00,None,None,None,None,None,True,True


# The `ActionValue` document

In [ ]:
#export

class ActionValues(mongoengine.Document):
    "Store the `ActionValue` document"
    ## event
    event_id = mongoengine.IntField(db_field="id")
    orig_event_id = mongoengine.StringField(db_field="eventId", primary_key=True)
    
    ## player
    player_id = mongoengine.IntField(db_field="player_id", required=False)
    player_name = mongoengine.StringField(
        db_field="player_name", required=False, default="Unknown"
    )
    
    ## opponent
    opponent_position_id = mongoengine.IntField(db_field="opponent_position_id", required=False)
    opponent_id = mongoengine.IntField(db_field="opponent_id", required=False) 
    opponent_team_name = mongoengine.StringField(db_field="opponent_team_name", required=False) 
    opponent_team_id = mongoengine.IntField(db_field="opponent_team_id", required=False)
    opponent_name = mongoengine.StringField(db_field="opponent_name", required=False)

    ## team
    team_id = mongoengine.IntField(db_field="team_id")
    team_name = mongoengine.StringField(db_field="team_name")
    
    ## possession
    possession_id = mongoengine.IntField(db_field="poss_id", required=True)
    possession_name = mongoengine.StringField(db_field="possession_name", required=True)
    possession_team_id = mongoengine.IntField(
        db_field="possession_team_id", required=True
    )
    possession_number = mongoengine.IntField(
        db_field="possession_number", required=True
    )
    seconds_since_poss = mongoengine.FloatField(
        db_field="seconds_since_poss", min_value=0.0, required=True
    )
    standart_name = mongoengine.StringField(db_field="standart_name", required=True)
    
    ## timing
    period_id = mongoengine.IntField(
        db_field="period_id", required=True, min_value=1, max_value=2
    )
    time_seconds = mongoengine.FloatField(db_field="time_seconds", required=True)
    minutes = mongoengine.IntField(
        db_field="min", required=True, min_value=0, max_value=130
    )
    sec = mongoengine.IntField(
        db_field="sec", required=True, min_value=0, max_value=59
    )

    ## attack
    attack_status_name = mongoengine.StringField(
        db_field="attack_status_name", required=True
    )
    attack_type_name = mongoengine.StringField(
        db_field="attack_type_name", required=True
    )
    attack_team_id = mongoengine.StringField(db_field="attack_team_id", required=True)

    ## actions
    action_id = mongoengine.IntField(db_field="action_id", required=True)
    type_name = mongoengine.StringField(
        db_field="type_name", required=True
    )
    result_id = mongoengine.IntField(
        db_field="result_id", required=True, min_value=-1, max_value=1
    )
    result_name = mongoengine.StringField(db_field="result_name", required=True)
    generic_action_type_id = mongoengine.IntField(
        db_field="generic_action_type_id", required=True
    )
    generic_action_type_name = mongoengine.StringField(
        db_field="generic_action_type_name", required=True
    )
    action_subtype_name = mongoengine.StringField(
        db_field="action_subtype_name", required=True
    )

    under_pressure = mongoengine.BooleanField(db_field="under_pressure", required=True)
    high_speed = mongoengine.BooleanField(db_field="high_speed", required=True)

    ## we allow extra meters to adjust pitch coordinates
    start_x = mongoengine.FloatField(db_field="start_x", min_value=-1, max_value=107)
    start_y = mongoengine.FloatField(db_field="start_y", min_value=-1, max_value=70)
    end_x = mongoengine.FloatField(db_field="end_x", min_value=-1, max_value=107)
    end_y = mongoengine.FloatField(db_field="end_y", min_value=-1, max_value=70)

    gate_x = mongoengine.FloatField(db_field="gate_x", required=False)
    gate_y = mongoengine.FloatField(db_field="gate_y", required=False)

    body_name = mongoengine.StringField(
        db_field="body_name", required=False, default="Unknown"
    )
    is_poss_team = mongoengine.BooleanField(db_field="is_poss_team", required=True)
    is_att_team = mongoengine.BooleanField(db_field="is_att_team", required=True)
    
    ## optional
    touches = mongoengine.StringField(db_field="touches", required=False)
    shot_type = mongoengine.StringField(db_field="shot_type", required=False)
    shot_handling = mongoengine.StringField(db_field="shot_handling", required=False)
    
    ## extra fields computed
    action_reference = mongoengine.StringField(
        db_field="action_reference", required=False
    )
    
    proba_goal = mongoengine.FloatField(db_field="proba_goal", required=True, min_value=0.0, max_value=1.0)
    proba_none = mongoengine.FloatField(db_field="proba_none", required=True, min_value=0.0, max_value=1.0)
    action_value = mongoengine.FloatField(db_field="action_value", required=False, min_value=-1.0, max_value=1.0)
    
    game_id = mongoengine.IntField(
        db_field="gameId", required=True
    )
    target = mongoengine.StringField(
        db_field="target", required=True, choices=("goal", "no-goal"))
    
    meta = {
        "db_alias": "inStat",
        "collection": CONFIG["connections"]["inStat"]["actionValues"],
    }